In [1]:
import cv2
import numpy as np
from typing import Tuple, Dict
import os

def detect_drain_contamination(image_path: str, debug: bool = False) -> Dict:
    """
    배수구 사진에서 이물질을 감지하여 오염도를 판별합니다.
    금속 격자는 오염물질로 간주하지 않습니다.
    
    Parameters:
    ----------
    image_path : str
        분석할 배수구 이미지 파일 경로
    debug : bool
        중간 과정을 시각화할지 여부
        
    Returns:
    -------
    result : dict
        {
            'contamination_ratio': float,     # 오염도 비율 (0.0 ~ 1.0)
            'contamination_level': str,       # 오염 등급 ('청결', '청소요망')
            'total_area': int,               # 전체 분석 영역 면적
            'contaminated_area': int,        # 오염된 영역 면적
            'debris_count': int,             # 감지된 이물질 개수
            'debris_types': list             # 감지된 이물질 유형들
        }
    """
    
    # 1. 이미지 로드 및 전처리
    img_original = cv2.imread(image_path)
    if img_original is None:
        raise FileNotFoundError(f"이미지를 찾을 수 없습니다: {image_path}")
    
    # 이미지 크기 조정 (처리 속도 향상)
    height, width = img_original.shape[:2]
    if width > 800:
        scale = 800 / width
        new_width = int(width * scale)
        new_height = int(height * scale)
        img_original = cv2.resize(img_original, (new_width, new_height))
    
    img_work = img_original.copy()
    
    # 2. 색상 공간 변환
    hsv = cv2.cvtColor(img_work, cv2.COLOR_BGR2HSV)
    lab = cv2.cvtColor(img_work, cv2.COLOR_BGR2LAB)
    gray = cv2.cvtColor(img_work, cv2.COLOR_BGR2GRAY)
    
    if debug:
        cv2.imshow("Original", img_original)
        cv2.imshow("Grayscale", gray)
        cv2.waitKey(0)
    
    # 3. 금속 격자 마스크 생성 (제외할 영역)
    metal_mask = create_metal_grate_mask(img_work, hsv, gray, debug)
    
    # 4. 이물질 감지를 위한 다중 접근법
    debris_masks = []
    debris_types = []
    
    # 4.1. 유기물 감지 (낙엽, 나뭇가지 등)
    organic_mask = detect_organic_debris(hsv, lab, debug)
    if np.sum(organic_mask) > 0:
        debris_masks.append(organic_mask)
        debris_types.append("유기물")
    
    # 4.2. 인공 쓰레기 감지 (플라스틱, 종이 등)
    plastic_mask = detect_artificial_debris(hsv, lab, debug)
    if np.sum(plastic_mask) > 0:
        debris_masks.append(plastic_mask)
        debris_types.append("인공쓰레기")
    
    # 4.3. 침전물/진흙 감지
    sediment_mask = detect_sediment(gray, lab, debug)
    if np.sum(sediment_mask) > 0:
        debris_masks.append(sediment_mask)
        debris_types.append("침전물")
    
    # 4.4. 어두운 막힘 영역 감지
    blockage_mask = detect_dark_blockage(gray, debug)
    if np.sum(blockage_mask) > 0:
        debris_masks.append(blockage_mask)
        debris_types.append("막힘")
    
    # 5. 전체 이물질 마스크 통합
    combined_debris_mask = np.zeros_like(gray, dtype=np.uint8)
    for mask in debris_masks:
        combined_debris_mask = cv2.bitwise_or(combined_debris_mask, mask)
    
    # 6. 금속 격자 영역 제외
    final_debris_mask = cv2.bitwise_and(combined_debris_mask, cv2.bitwise_not(metal_mask))
    
    # 7. 노이즈 제거
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    final_debris_mask = cv2.morphologyEx(final_debris_mask, cv2.MORPH_OPEN, kernel)
    final_debris_mask = cv2.morphologyEx(final_debris_mask, cv2.MORPH_CLOSE, kernel)
    
    # 8. 이물질 개수 세기
    contours, _ = cv2.findContours(final_debris_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    debris_count = len([cnt for cnt in contours if cv2.contourArea(cnt) > 50])
    
    # 9. 오염도 계산
    total_area = gray.shape[0] * gray.shape[1]
    contaminated_area = np.sum(final_debris_mask > 0)
    contamination_ratio = contaminated_area / total_area
    
    # 10. 오염 등급 결정
    contamination_level = classify_contamination_level(contamination_ratio, debris_count)
    
    # 11. 디버그 시각화
    if debug:
        visualize_results(img_original, metal_mask, final_debris_mask, debris_masks, debris_types)
    
    return {
        'contamination_ratio': contamination_ratio,
        'contamination_level': contamination_level,
        'total_area': total_area,
        'contaminated_area': contaminated_area,
        'debris_count': debris_count,
        'debris_types': list(set(debris_types))
    }


def create_metal_grate_mask(img: np.ndarray, hsv: np.ndarray, gray: np.ndarray, debug: bool = False) -> np.ndarray:
    """금속 격자 영역 마스크 생성"""
    # 금속의 특징: 회색~검은색, 높은 대비, 직선적 구조
    
    # 1. 색상 기반 금속 감지
    # 금속은 보통 낮은 채도를 가짐
    metal_color_mask = cv2.inRange(hsv, (0, 0, 30), (180, 50, 200))
    
    # 2. 에지 검출로 격자 구조 찾기
    edges = cv2.Canny(gray, 50, 150)
    
    # 3. 직선 검출 (Hough Transform)
    lines = cv2.HoughLinesP(edges, 1, np.pi/180, threshold=30, minLineLength=20, maxLineGap=10)
    
    line_mask = np.zeros_like(gray)
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(line_mask, (x1, y1), (x2, y2), 255, 3)
    
    # 4. 금속 격자 마스크 결합
    metal_mask = cv2.bitwise_or(metal_color_mask, line_mask)
    
    # 5. 모폴로지 연산으로 격자 구조 강화
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    metal_mask = cv2.morphologyEx(metal_mask, cv2.MORPH_CLOSE, kernel)
    
    if debug:
        cv2.imshow("Metal Grate Mask", metal_mask)
        cv2.waitKey(0)
    
    return metal_mask


def detect_organic_debris(hsv: np.ndarray, lab: np.ndarray, debug: bool = False) -> np.ndarray:
    """유기물 이물질 감지 (낙엽, 나뭇가지 등)"""
    # 갈색, 노란색, 녹색 계열의 유기물 감지
    
    # 갈색 범위 (낙엽)
    brown_lower = np.array([10, 50, 50])
    brown_upper = np.array([25, 255, 200])
    brown_mask = cv2.inRange(hsv, brown_lower, brown_upper)
    
    # 황색 범위 (마른 잎)
    yellow_lower = np.array([20, 50, 50])
    yellow_upper = np.array([35, 255, 255])
    yellow_mask = cv2.inRange(hsv, yellow_lower, yellow_upper)
    
    # 녹색 범위 (푸른 잎)
    green_lower = np.array([35, 40, 40])
    green_upper = np.array([85, 255, 255])
    green_mask = cv2.inRange(hsv, green_lower, green_upper)
    
    # 통합
    organic_mask = cv2.bitwise_or(cv2.bitwise_or(brown_mask, yellow_mask), green_mask)
    
    if debug:
        cv2.imshow("Organic Debris", organic_mask)
        cv2.waitKey(0)
    
    return organic_mask


def detect_artificial_debris(hsv: np.ndarray, lab: np.ndarray, debug: bool = False) -> np.ndarray:
    """인공 쓰레기 감지 (플라스틱, 종이 등)"""
    # 밝은 색상, 높은 채도의 인공물 감지
    
    # 빨간색 계열
    red_lower1 = np.array([0, 50, 50])
    red_upper1 = np.array([10, 255, 255])
    red_lower2 = np.array([170, 50, 50])
    red_upper2 = np.array([180, 255, 255])
    red_mask = cv2.bitwise_or(cv2.inRange(hsv, red_lower1, red_upper1),
                              cv2.inRange(hsv, red_lower2, red_upper2))
    
    # 파란색 계열
    blue_lower = np.array([100, 50, 50])
    blue_upper = np.array([130, 255, 255])
    blue_mask = cv2.inRange(hsv, blue_lower, blue_upper)
    
    # 흰색 계열 (종이, 플라스틱)
    white_lower = np.array([0, 0, 200])
    white_upper = np.array([180, 30, 255])
    white_mask = cv2.inRange(hsv, white_lower, white_upper)
    
    # 통합
    artificial_mask = cv2.bitwise_or(cv2.bitwise_or(red_mask, blue_mask), white_mask)
    
    if debug:
        cv2.imshow("Artificial Debris", artificial_mask)
        cv2.waitKey(0)
    
    return artificial_mask


def detect_sediment(gray: np.ndarray, lab: np.ndarray, debug: bool = False) -> np.ndarray:
    """침전물/진흙 감지"""
    # 어두운 갈색, 회색 계열의 침전물
    
    # 어두운 영역 (그러나 완전히 검은색은 아닌)
    _, sediment_mask = cv2.threshold(gray, 80, 255, cv2.THRESH_BINARY_INV)
    
    # 텍스처 기반 필터링 (침전물은 보통 거친 텍스처를 가짐)
    kernel = np.ones((5, 5), np.float32) / 25
    smooth = cv2.filter2D(gray, -1, kernel)
    texture = cv2.absdiff(gray, smooth)
    _, texture_mask = cv2.threshold(texture, 15, 255, cv2.THRESH_BINARY)
    
    # 침전물 마스크
    sediment_mask = cv2.bitwise_and(sediment_mask, texture_mask)
    
    if debug:
        cv2.imshow("Sediment", sediment_mask)
        cv2.waitKey(0)
    
    return sediment_mask


def detect_dark_blockage(gray: np.ndarray, debug: bool = False) -> np.ndarray:
    """어두운 막힘 영역 감지"""
    # 매우 어두운 영역 (완전히 막힌 부분)
    _, blockage_mask = cv2.threshold(gray, 40, 255, cv2.THRESH_BINARY_INV)
    
    # 모폴로지 연산으로 노이즈 제거
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    blockage_mask = cv2.morphologyEx(blockage_mask, cv2.MORPH_OPEN, kernel)
    
    if debug:
        cv2.imshow("Dark Blockage", blockage_mask)
        cv2.waitKey(0)
    
    return blockage_mask


def classify_contamination_level(ratio: float, debris_count: int) -> str:
    """오염도 등급 분류 (2단계)"""
    # 오염도 비율이 10% 미만이고 이물질이 5개 미만이면 청결
    if ratio < 0.10 and debris_count < 5:
        return "청결"
    else:
        return "청소요망"


def visualize_results(img: np.ndarray, metal_mask: np.ndarray, debris_mask: np.ndarray, 
                     debris_masks: list, debris_types: list, debug: bool = True):
    """결과 시각화"""
    if not debug:
        return
        
    # 결과 이미지 생성
    result_img = img.copy()
    
    # 금속 격자는 파란색으로 표시 (제외된 영역)
    result_img[metal_mask > 0] = [255, 0, 0]  # 파란색
    
    # 이물질은 빨간색으로 표시
    result_img[debris_mask > 0] = [0, 0, 255]  # 빨간색
    
    # 투명도 적용
    overlay = img.copy()
    overlay[metal_mask > 0] = [255, 0, 0]
    overlay[debris_mask > 0] = [0, 0, 255]
    result_img = cv2.addWeighted(img, 0.7, overlay, 0.3, 0)
    
    cv2.imshow("Analysis Result (Blue: Metal Grate, Red: Debris)", result_img)
    cv2.imshow("Final Debris Mask", debris_mask)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


# 사용 예시
if __name__ == "__main__":
    # 이미지 경로 설정
    image_path = "C:/Users/USER/Desktop/final/0609 data/final_heavy_aug_290/heavy_28_crop0_aug4.jpg"  # 실제 경로로 변경
  
    try:
        # 오염도 분석 실행
        result = detect_drain_contamination(image_path, debug=True)
        
        # 결과 출력
        print("=" * 50)
        print("배수구 오염도 분석 결과")
        print("=" * 50)
        print(f"오염도 비율      : {result['contamination_ratio']:.3f} ({result['contamination_ratio']*100:.1f}%)")
        print(f"오염 등급        : {result['contamination_level']}")
        print(f"전체 영역        : {result['total_area']:,} 픽셀")
        print(f"오염된 영역      : {result['contaminated_area']:,} 픽셀")
        print(f"이물질 개수      : {result['debris_count']}개")
        print(f"감지된 이물질 유형: {', '.join(result['debris_types'])}")
        print("=" * 50)
        
        # 권장사항 출력
        level = result['contamination_level']
        if level == "청결":
            print("권장사항: 현재 상태가 양호합니다. 정기적인 점검을 권장합니다.")
        else:  # 청소요망
            print("권장사항: 청소가 필요합니다. 이물질을 제거해 주세요.")
            
    except FileNotFoundError as e:
        print(f"오류: {e}")
    except Exception as e:
        print(f"분석 중 오류가 발생했습니다: {e}")

배수구 오염도 분석 결과
오염도 비율      : 0.107 (10.7%)
오염 등급        : 청소요망
전체 영역        : 176,000 픽셀
오염된 영역      : 18,823 픽셀
이물질 개수      : 12개
감지된 이물질 유형: 인공쓰레기, 유기물, 침전물, 막힘
권장사항: 청소가 필요합니다. 이물질을 제거해 주세요.
